In [1]:
!sudo rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/r1.0
!sudo pip3 install --upgrade pip
!sudo pip3 install shapely
import os
import tensorflow as tf
import math
import numpy as np
import itertools
print("Version is")
print(tf.__version__)
tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
from shapely.geometry import Polygon, LineString
import random
import sys

Cloning into 'waymo-od'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 516 (delta 91), reused 95 (delta 52), pack-reused 363
Receiving objects: 100% (516/516), 14.06 MiB | 18.55 MiB/s, done.
Resolving deltas: 100% (283/283), done.
* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/master
  remotes/origin/r1.0
  remotes/origin/r1.0-tf1.15
  remotes/origin/r1.0-tf2.0
Note: checking out 'remotes/origin/r1.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a66eb0a... Merge branch 'master' into r1.0
Requi

In [2]:
!gsutil ls gs://cien-9101-19fall-waymo/ > files.txt

In [4]:
with open('files.txt') as f:
    lines = f.read().splitlines()

In [12]:
str1 = "!gsutil cp " + lines[0] + " " + "./dataset/hi.tfrecord"
print(str1)

!gsutil cp gs://cien-9101-19fall-waymo/training_0000_segment-14143054494855609923_4529_100_4549_100_with_camera_labels.tfrecord ./dataset/hi.tfrecord


32512

In [5]:
TYPE_VEHICLE = 1
FRONT = 1
FPS = 10
VERIFY_THRESHOLD = 0.05

#Eager execution is enabled by default in TF 2.0 Therefore we need not enable it explicitly.
#tf.enable_eager_execution()


total_frames = 0
skipped_frames = 0

"""
features:
[vx, vy, vz, dx, dy, vfx, vfy, vfz, afx, afy, afz]
labels:
[ax, ay, az]
"""

def write_to_csv(filename, feats, labels):
    comb_np = np.hstack((feats, labels))
    np.savetxt(filename, comb_np, delimiter=",")


def collect_vehicle_laser_labels(frame):
    v_laser_labels = []

    for ll in frame.laser_labels:
        v_laser_labels.append(ll)

    return v_laser_labels


def get_vehicle_pose(frame):
    pose = []
    for image in frame.images:
        pose = [t for t in image.pose.transform]

    return np.asarray(pose).reshape((4,4))


def get_current_car_velocity_wrt_GF(frame):
    """
    Return the speed v_x and v_y of the current car
    """
    for image in frame.images:
        return np.asarray([image.velocity.v_x, image.velocity.v_y, image.velocity.v_z])

    return None


def get_front_car_velocity_wrt_GF(front_car_label, vehicle_pose, v_cur_GF):
    v_front_VF = np.asarray([front_car_label.metadata.speed_x, front_car_label.metadata.speed_y, 0])
    _v_front_VF = np.hstack((v_front_VF, [0])) # padded 0 for matrix multiplication
    return np.matmul(vehicle_pose, _v_front_VF)[:3] - v_cur_GF


def get_relative_distance(front_car_label):
    return np.asarray([front_car_label.box.center_x, front_car_label.box.center_y])


def get_current_car_accel_GF_per_frame(dt, v_cur_GF, v_cur_GF_prev):
    return cal_aceleration(v_cur_GF, v_cur_GF_prev, dt) if v_cur_GF_prev is not None else np.asarray([0, 0, 0])


def get_front_car_GF_features_per_frame(dt, frame, vehicle_pose, front_car_label,
                                        v_cur_GF, v_front_GF_prev, verify=False):

    if verify and random.random() < VERIFY_THRESHOLD:
        verify_front_car_label(frame, front_car_label)

    relative_dist = get_relative_distance(front_car_label) # 2 * 1
    v_front_GF = get_front_car_velocity_wrt_GF(front_car_label, vehicle_pose, v_cur_GF) # 3 * 1
    a_front_GF = cal_aceleration(v_front_GF, v_front_GF_prev, dt) if v_front_GF_prev is not None else np.asarray([0, 0, 0]) # 3 * 1

    return np.hstack((relative_dist, v_front_GF, a_front_GF)), v_front_GF


def get_essentials_per_frame(dt, frame, front_car_label, v_cur_GF_prev, v_front_GF_prev):
    vehicle_pose = get_vehicle_pose(frame)
    v_cur_GF = get_current_car_velocity_wrt_GF(frame) # 3 * 1
    front_GF_feat, v_front_GF = get_front_car_GF_features_per_frame(dt, frame, vehicle_pose, front_car_label,
                                                                  v_cur_GF, v_front_GF_prev) # 8 * 1
    a_cur_GF = get_current_car_accel_GF_per_frame(dt, v_cur_GF, v_cur_GF_prev) # 3 * 1

    return np.hstack((v_cur_GF, front_GF_feat)), a_cur_GF, v_cur_GF, v_front_GF


def get_features_and_labels(frames):
    global total_frames
    global skipped_frames

    feat_set = []
    label_set = []

    dt = len(frames) * 1.0 / FPS / (len(frames) - 1)

    # init
    v_cur_GF_prev = None
    v_front_GF_prev = None

    for frame in frames:
        total_frames += 1

        # Capture the front car
        v_laser_labels = collect_vehicle_laser_labels(frame)
        front_car_label = get_front_car_laser_label(v_laser_labels)

        if front_car_label is None:
            skipped_frames += 1
            print("No front car captured, will skip this frame")
            continue

        feats, labels, v_cur_GF_prev, v_front_GF_prev = get_essentials_per_frame(dt, frame, front_car_label, v_cur_GF_prev, v_front_GF_prev)
        feat_set.append(feats)
        label_set.append(labels)

    return np.asarray(feat_set), np.asarray(label_set)

def intersects(label):
  # Starting from the upper-left corner, clock direction
  bounding_box = Polygon([
      (label.box.center_x - 0.5 * label.box.length, label.box.center_y + 0.5 * label.box.width), 
      (label.box.center_x + 0.5 * label.box.length, label.box.center_y - 0.5 * label.box.width), 
      (label.box.center_x + 0.5 * label.box.length, label.box.center_y + 0.5 * label.box.width),
      (label.box.center_x - 0.5 * label.box.length, label.box.center_y - 0.5 * label.box.width)
  ])
  
  line = LineString([(0, 0), (label.box.center_x, 0)])
  
  return bounding_box.intersects(line)


def get_front_car_laser_label(labels):
    """
    Find the closest bounding box which intersects with y = 0 and its center_x is positive
    """

    front_car_label = None

    for label in labels:
        if label.box.center_x < 0:
            continue

        if intersects(label):
            if front_car_label is None or front_car_label.box.center_x > label.box.center_x:
                front_car_label = label

    return front_car_label


def cal_aceleration(v1, v2, dt):
    return (v2 - v1) / dt;

In [16]:
import os
from os import listdir
from os.path import isfile, join

seg_path = "./dataset"
result_path = "data.csv"

files = [join(seg_path, f) for f in listdir(seg_path) if isfile(join(seg_path, f))]

print("Total files: ", len(files))

feat_set = None
label_set = None

for i in range(len(files)):
    file = files[i]
    print(file)
    dataset = tf.data.TFRecordDataset(file, compression_type='')

    # Load frames from dataset
    frames = []
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        frames.append(frame)

    print("filename:", file, "Num of frames:", len(frames))

    feats, labels = get_features_and_labels(frames)

    feat_set = np.vstack((feat_set, feats)) if feat_set is not None else feats
    label_set = np.vstack((label_set, labels)) if label_set is not None else labels

    print("Progress: ", i, "/", len(files))

write_to_csv(result_path, feat_set, label_set)
print("Finished! Total_frames = ", total_frames, " skipped_frames = ", skipped_frames)

Total files:  125
./dataset/validation_0003_segment-14931160836268555821_5778_870_5798_870.tfrecord
filename: ./dataset/validation_0003_segment-14931160836268555821_5778_870_5798_870.tfrecord Num of frames: 198
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this fra

No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip

filename: ./dataset/validation_0000_segment-14956919859981065721_1759_980_1779_980_with_camera_labels.tfrecord Num of frames: 199
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip t

filename: ./dataset/validation_0003_segment-16229547658178627464_380_000_400_000.tfrecord Num of frames: 197
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front ca

No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip

...

No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
Progress:  101 / 125
./dataset/training_0005_segment-11318901554551149504_520_000_540_000.tfrecord
filename: ./dataset/training_0005_segment-11318901554551149504_520_000_540_000.tfrecord Num of frames: 199
Progress:  102 / 125
./dataset/training_0023_segment-4575961016807404107_880_000_900_000.tfrecord
filename: ./dataset/training_0023_segment-4575961016807404107_880_000_900_000.tfrecord Num of frames: 198
Progress:  103 / 125
./dataset/training_0016_segment-17850487901509155700_9065_000_9085_000.tfrecord
filename: ./dataset/training_0016_segment-17850487901509155700_9065_000_9085_000.tfrecord Num of frames: 198
Progress:  104 / 125
./dataset/training_0008_segment-13182548552824592684_4160_250_4180_250.tfrecord
filename: ./dataset/training_0008_segment-13182548552824592

filename: ./dataset/training_0025_segment-5835049423600303130_180_000_200_000.tfrecord Num of frames: 198
Progress:  113 / 125
./dataset/training_0006_segment-12337317986514501583_5346_260_5366_260.tfrecord
filename: ./dataset/training_0006_segment-12337317986514501583_5346_260_5366_260.tfrecord Num of frames: 198
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, w

filename: ./dataset/training_0011_segment-14810689888487451189_720_000_740_000.tfrecord Num of frames: 198
Progress:  115 / 125
./dataset/training_0000_segment-14143054494855609923_4529_100_4549_100_with_camera_labels.tfrecord
filename: ./dataset/training_0000_segment-14143054494855609923_4529_100_4549_100_with_camera_labels.tfrecord Num of frames: 198
Progress:  116 / 125
./dataset/training_0009_segment-13823509240483976870_1514_190_1534_190.tfrecord
filename: ./dataset/training_0009_segment-13823509240483976870_1514_190_1534_190.tfrecord Num of frames: 198
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this f

No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip

No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip this frame
No front car captured, will skip